# Difference snow depth estimates from SkySat and Lidar

In [ ]:
import os
import matplotlib.pyplot as plt
import xdem
import geoutils as gu
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
data_dir = '/Volumes/LaCie/raineyaberle/Research/PhD/SkySat-Stereo/study-sites/MCS'
skysat_sd_fn = os.path.join(data_dir, '20240420', 'skysat_snow_nofilter', 'ddem_from_original_DEM.tif')
snow_mask_fn = os.path.join(data_dir, '20240420', 'skysat_snow_nofilter', 'land_cover_masks', 'snow_mask.tif')
refdem_fn = os.path.join(data_dir, 'refdem', 'MCS_REFDEM_WGS84.tif')
lidar_sd_fn = os.path.join(data_dir, 'SNEX_MCS_Lidar', 'SNEX_MCS_Lidar_20240418_SD_V01.0.tif')
out_dir = os.path.join(data_dir, '20240420', 'skysat-lidar_nofilter')

# Load input files
skysat_sd = xdem.DEM(skysat_sd_fn)
refdem = xdem.DEM(refdem_fn).reproject(skysat_sd)
lidar_sd = xdem.DEM(lidar_sd_fn).reproject(skysat_sd)
# snow_mask = gu.Raster(snow_mask_fn, load_data=True).reproject(skysat_sd)
# snow_mask = (snow_mask==1)
# # Mask pixels not snow-covered
# skysat_sd.set_mask(~snow_mask)

In [ ]:
fig, ax = plt.subplots(2, 3, figsize=(12,8), gridspec_kw=dict(height_ratios=[2,1]))
vmin, vmax = -2, 5
bins = np.linspace(vmin, vmax, 50)
for i, sd in enumerate([skysat_sd, lidar_sd]):
    # map
    sd.plot(ax=ax[0,i], cmap='Blues', vmin=0, vmax=vmax)
    ax[0,i].set_xticks([])
    ax[0,i].set_yticks([])
    # histogram
    hist = ax[1,i].hist(sd.data.ravel(), bins=bins)
    median = np.nanmedian(sd)
    ax[1,i].axvline(x=median, ymin=0, ymax=np.nanmax(hist[0])*1.2, color='k')
    ax[1,i].set_ylim(0, np.nanmax(hist[0])*1.2)
    ax[1,i].set_xlabel('Snow depths [m]')
    ax[1,i].set_xlim(vmin, vmax)
ax[0,0].set_title('SkySat')
ax[0,1].set_title('Lidar')
# Difference
diff = skysat_sd - lidar_sd
diff.plot(ax=ax[0,2], cmap='coolwarm_r', vmin=-3, vmax=3)
ax[1,2].hist(diff.data.ravel(), bins=np.linspace(-3, 3, 50))
ax[1,2].set_xlabel('Difference [m]')
fig.tight_layout()
plt.show()

# Save figure
fig_fn = os.path.join(out_dir, 'skysat-lidar.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
# Plot the difference as a function of terrain parameters
refdem_downsamp = refdem.reproject(res=10)
slope = refdem_downsamp.slope()
slope = slope.reproject(skysat_sd)
aspect = refdem_downsamp.aspect()
aspect = aspect.reproject(skysat_sd)

# Compile all stats into dataframe
terrain_cols = ['elevation', 'slope', 'aspect']
stats_df = pd.DataFrame(columns=terrain_cols + ['dDEM'])
for raster, col in zip([refdem, slope, aspect], terrain_cols):
    stats_df[col] = raster.data.ravel()
    # Create bins for column
    stats_df[col + '_bin'] = pd.cut(stats_df[col], bins=25, precision=0)
stats_df['dDEM'] = diff.data.ravel() 
stats_df.dropna(inplace=True)
stats_df.reset_index(drop=True, inplace=True)

# Plot
fig, ax = plt.subplots(1,3, figsize=(16,6))
for i, col in enumerate(terrain_cols):
    sns.boxplot(stats_df, x=col + '_bin', y='dDEM', showfliers=False, ax=ax[i])
    ax[i].set_title(col)
    if i==0:
        ax[i].set_ylabel('dDEM [m]')
    ax[i].set_xlabel(col)
    ax[i].set_xticklabels(ax[i].get_xticklabels(), rotation=90)
    ax[i].axhline(0, color='k')
fig.tight_layout()

# Save figure
fig_fn = os.path.join(out_dir, 'skysat-lidar_terrain_params.png')
fig.savefig(fig_fn, dpi=300, bbox_inches='tight')
print('Figure saved to file:', fig_fn)

In [ ]:
slope.plot(vmin=0, vmax=45)

In [ ]:
skysat_sd_norm = skysat_sd / np.nanmax(skysat_sd.data)
lidar_sd_norm = lidar_sd / np.nanmax(lidar_sd.data)
diff_norm = skysat_sd_norm - lidar_sd_norm
print(f'Median normalized difference = {np.nanmedian(diff_norm.data)}')

fig, ax = plt.subplots(2, 1, figsize=(6,10), gridspec_kw=dict(height_ratios=[2,1]))
diff_norm.plot(cmap='coolwarm_r', vmin=-0.2, vmax=0.2, ax=ax[0])
ax[1].hist(diff_norm.data.ravel(), bins=np.linspace(-0.2, 0.2, 50))
plt.show()

In [ ]:
()